# SwissHouseRSlaTank notebook example

In this example, the usage of the model "SwissHouseRad-v0" is demonstrated.

First, we import Energym and create the simulation environment by specifying the model, a weather file and the number of simulation days.

In [1]:
import sys
sys.path.insert(0, "/home/lesong/energym/")
import energym

weather = "CH_BS_Basel"
env = energym.make("SwissHouseRSlaTank-v0", weather=weather, simulation_days=20)

the initial variables are {'uHP': 0, 'uRSla': 0}


The control inputs can be inspected using the `get_inputs_names()` method.

In [2]:
inputs = env.get_inputs_names()
outputs = env.get_outputs_names()
print("inputs:", inputs)
print("outputs:", outputs)

inputs: ['uHP', 'uRSla']
outputs: ['TOut.T', 'heaPum.COP', 'heaPum.COPCar', 'heaPum.P', 'heaPum.QCon_flow', 'heaPum.QEva_flow', 'heaPum.TConAct', 'heaPum.TEvaAct', 'preHea.Q_flow', 'sla.QTot', 'sla.heatPortEmb[1].T', 'sla.m_flow', 'tanSH.heaPorSid.T', 'temRoo.T', 'weaBus.HDifHor', 'weaBus.HDirNor', 'weaBus.HGloHor', 'weaBus.HHorIR']


In [3]:
[ p for p in env.vrs if "tem" in p]

['tanSH.indTanHex.hAPipIns[1].temperatureDependent',
 'tanSH.indTanHex.hAPipIns[2].temperatureDependent',
 'tanSH.indTanHex.hAPipIns[3].temperatureDependent',
 'tanSH.indTanHex.hAPipIns[4].temperatureDependent',
 'tanSH.indTanHex.hAPipIns[5].temperatureDependent',
 'tanSH.indTanHex.hAPipIns[6].temperatureDependent',
 'tanSH.indTanHex.hA_temperatureDependent',
 'tanSH.indTanHex.temSenHex[1].T',
 'tanSH.indTanHex.temSenHex[1].port.Q_flow',
 'tanSH.indTanHex.temSenHex[1].port.T',
 'tanSH.indTanHex.temSenHex[2].T',
 'tanSH.indTanHex.temSenHex[2].port.Q_flow',
 'tanSH.indTanHex.temSenHex[2].port.T',
 'tanSH.indTanHex.temSenHex[3].T',
 'tanSH.indTanHex.temSenHex[3].port.Q_flow',
 'tanSH.indTanHex.temSenHex[3].port.T',
 'tanSH.indTanHex.temSenHex[4].T',
 'tanSH.indTanHex.temSenHex[4].port.Q_flow',
 'tanSH.indTanHex.temSenHex[4].port.T',
 'tanSH.indTanHex.temSenHex[5].T',
 'tanSH.indTanHex.temSenHex[5].port.Q_flow',
 'tanSH.indTanHex.temSenHex[5].port.T',
 'tanSH.indTanHex.temSenHex[6].T',
 't

To run the simulation, a number of steps is specified (here 288 steps per day for 10 days), a control input is specified and passed to the simulation model with the `step()` method. To generate some plots later on, we save all the outputs in lists.

In [4]:
from scipy import signal
import math

steps = 288*5
out_list = []
outputs = env.get_output()
controls = []
hour = 0
for i in range(steps):
    # control = {}
    # control['uHP'] = [0.5*(signal.square(0.1*i)+1.0)]
    # control['uRSla'] = [0.5*(math.sin(0.01*i)+1.0)]
    control = env.sample_random_action()
    controls +=[ {p:control[p][0] for p in control} ]
    outputs = env.step(control)
    _,hour,_,_ = env.get_date()
    out_list.append(outputs)

[OK] <JMIRuntime><value name="build_date">"Feb 24 2021"</value> <value name="build_time">"21:25:41"</value></JMIRuntime>
[WARNING] <ModelicaMessage category="warning"><value name="msg">"... loading ""tab1"" from ""/home/lesong/Energym_runs/1670810359_1654158/resources/0/weather.mos""&##10;"</value></ModelicaMessage>
[WARNING] <ModelicaMessage category="warning"><value name="msg">"... loading ""tab1"" from ""/home/lesong/Energym_runs/1670810359_1654158/resources/0/weather.mos""&##10;"</value></ModelicaMessage>


Since the outputs are given as dictionaries and are collected in lists, we can simply load them as a pandas.DataFrame.

In [5]:
import pandas as pd
out_df = pd.DataFrame(out_list)
cmd_df = pd.DataFrame(controls)

In [6]:
cmd_df

,uHP,uRSla
0,0.682025,0.453100
1,0.566132,0.565846
2,0.602041,0.807045
3,0.562501,0.356056
4,0.938982,0.924818
...,...,...
1435,0.581956,0.677956
1436,0.368417,0.220496
1437,0.297671,0.557814
1438,0.507737,0.023381


In [7]:
out_df

,TOut.T,heaPum.COP,heaPum.COPCar,heaPum.P,heaPum.QCon_flow,heaPum.QEva_flow,heaPum.TConAct,heaPum.TEvaAct,preHea.Q_flow,sla.QTot,sla.heatPortEmb[1].T,sla.m_flow,tanSH.heaPorSid.T,temRoo.T,weaBus.HDifHor,weaBus.HDirNor,weaBus.HGloHor,weaBus.HHorIR,time
0,273.040697,3.192122,10.146626,682.024956,2177.107074,-1495.082119,297.110338,267.828651,0.0,226.478321,293.192701,0.453100,293.305948,293.136082,0.0,0.0,0.0,198.356554,300.0
1,273.050842,3.297621,10.481968,566.131532,1866.886967,-1300.755435,296.834920,268.516295,0.0,456.855340,293.238278,0.565846,293.415632,293.124064,0.0,0.0,0.0,198.314931,600.0
2,273.060017,3.254962,10.346372,602.040708,1959.619736,-1357.579028,297.036738,268.327473,0.0,715.303308,293.292800,0.807045,293.504631,293.113974,0.0,0.0,0.0,198.276953,900.0
3,273.067805,3.287581,10.450055,562.501192,1849.268090,-1286.766898,297.003347,268.582124,0.0,565.192775,293.244268,0.356056,293.612221,293.102970,0.0,0.0,0.0,198.244444,1200.0
4,273.073788,2.959732,9.407939,938.982368,2779.136064,-1840.153697,298.374321,266.659159,0.0,1220.650072,293.401461,0.924818,293.744302,293.096298,0.0,0.0,0.0,198.219227,1500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,278.738889,3.449410,10.964451,581.956148,2007.405124,-1425.448976,301.245128,273.770418,0.0,2343.550760,296.846855,0.677956,297.682927,296.260967,0.0,0.0,0.0,298.691478,430800.0
1436,278.737656,3.731032,11.859628,368.416786,1374.574636,-1006.157850,300.574764,275.230397,0.0,1126.277353,296.537821,0.220496,297.730278,296.256252,0.0,0.0,0.0,298.764373,431100.0
1437,278.743194,3.854974,12.253598,297.671467,1147.515825,-849.844358,300.286650,275.780651,0.0,2054.646902,296.772461,0.557814,297.667113,296.258799,0.0,0.0,0.0,298.817684,431400.0
1438,278.756858,3.532416,11.228299,507.737100,1793.538621,-1285.801520,301.089306,274.274087,0.0,149.619500,296.284420,0.023381,297.795671,296.247015,0.0,0.0,0.0,298.847523,431700.0


To generate plots, we can directly get the data from the DataFrames, by using the key names. Displayed are the room temperature, the supply temperature and the return temperature, as well as the external temperature, and the heat pump energy.

In [8]:
import matplotlib.pyplot as plt
%matplotlib inline

f, (ax1,ax2,ax3) = plt.subplots(3,figsize=(10,15))#


ax1.plot(out_df['temRoo.T']-273.15, 'r')
ax1.plot(out_df['sla.heatPortEmb[1].T']-273.15, 'b--')
ax1.plot(out_df['heaPum.TEvaAct']-273.15, 'orange')
ax1.set_ylabel('Temp')
ax1.set_xlabel('Steps')

ax2.plot(out_df['TOut.T']-273.15, 'r')
ax2.set_ylabel('Temp')
ax2.set_xlabel('Steps')

ax3.plot(out_df['heaPum.QCon_flow'], 'g')
ax3.set_ylabel('Energy')
ax3.set_xlabel('Steps')

plt.subplots_adjust(hspace=0.4)

plt.show()

<IPython.core.display.Javascript object>

To end the simulation, the `close()` method is called. It deletes files that were produced during the simulation and stores some information about the simulation in the *energym_runs* folder.

In [9]:
env.close()